##### 실습: 삼성전자의 주가 데이터를 가져와서 분석하는 ChatGPT 어플리케이션을 작성하시오(힌트: pykrx 패키지)

In [24]:
import os
import openai
from pykrx import stock
from datetime import datetime, timedelta
import pandas as pd

# 1. OpenAI API 키 불러오기
openai.api_key = os.getenv("OPENAI_API_KEY")

# 2. 삼성전자 주가 데이터 가져오기
def get_samsung_data():
    end = datetime.today()
    start = end - timedelta(days=365)
    df = stock.get_market_ohlcv_by_date(start.strftime('%Y%m%d'),
                                        end.strftime('%Y%m%d'),
                                        "005930")
    df.reset_index(inplace=True)
    return df

# 3. GPT에게 질문 전달 (스트리밍 응답)
def ask_chatgpt_with_stock(df, user_question):
    summary_text = f"""
[삼성전자 최근 1년 주가 요약]
- 기간: {df['날짜'].min().date()} ~ {df['날짜'].max().date()}
- 최고가: {df['고가'].max():,}원
- 최저가: {df['저가'].min():,}원
- 평균 종가: {df['종가'].mean():,.2f}원
- 최근 5일 종가:
{df[['날짜', '종가']].tail().to_string(index=False)}
"""

    gen = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": f"""
                삼성전자의 주가 데이터를 가져와서 분석하는 ChatGPT 어플리케이션입니다. 
                아래 데이터를 참고해서 사용자 질문에 대답하세요.

                {summary_text}
                """
            },
            {
                "role": "user",
                "content": user_question
            }
        ],
        temperature=0,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stream=True
    )

    print("ChatGPT 응답:")
    for response in gen:
        delta = response.choices[0].delta
        if delta.content:
            print(delta.content, end='')

# 4. 실행
if __name__ == "__main__":
    df = get_samsung_data()
    print("삼성전자 주가 데이터가 준비되었습니다.")

    # 질문 직접 지정
    question = "최근 1년간 삼성전자 주가 추세를 알려줘"

    ask_chatgpt_with_stock(df, question)


삼성전자 주가 데이터가 준비되었습니다.
ChatGPT 응답:
최근 1년간 삼성전자의 주가는 다음과 같은 추세를 보였습니다:

- **최고가**: 86,100원
- **최저가**: 49,900원
- **평균 종가**: 60,088.38원

최근 5일간의 종가는 다음과 같습니다:
- 2025-07-15: 63,700원
- 2025-07-16: 64,700원
- 2025-07-17: 66,700원
- 2025-07-18: 67,100원
- 2025-07-21: 67,800원

이 데이터를 통해 볼 때, 최근 1년간 삼성전자의 주가는 전반적으로 상승세를 보였으며, 특히 최근 5일간의 종가는 지속적으로 상승하고 있는 모습을 확인할 수 있습니다.

In [28]:
import os
import openai
from pykrx import stock
from datetime import datetime, timedelta

# API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# 삼성전자 주가 데이터 가져오기
end = datetime.today()
start = end - timedelta(days=365)
df = stock.get_market_ohlcv_by_date(start.strftime('%Y%m%d'), end.strftime('%Y%m%d'), "005930")
df.reset_index(inplace=True)

# 요약 텍스트 구성
summary = f"""
[삼성전자 최근 1년 주가 요약]
- 기간: {df['날짜'].min().date()} ~ {df['날짜'].max().date()}
- 최고가: {df['고가'].max():,}원
- 최저가: {df['저가'].min():,}원
- 평균 종가: {df['종가'].mean():,.2f}원
- 최근 5일 종가:
{df[['날짜', '종가']].tail().to_string(index=False)}
"""

# GPT 호출
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": """"
            너는 금융 전문가야. 아래의 주가 데이터를 바탕으로 간단한 분석을 해줘.
            결과를 보기에 깔끔하게 표로 정리해서 알려줘
            """
        },
        {
            "role": "user",
            "content": summary + "\n\nQ: 최근 1년간 삼성전자 주가 흐름을 분석해줘."
        }
    ],
    temperature=0,
    max_tokens=512,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stream=False
)

# 결과 출력
print("ChatGPT 분석 결과:")
print(response.choices[0].message.content)


ChatGPT 분석 결과:
삼성전자의 최근 1년간 주가 흐름을 분석한 결과를 아래의 표로 정리하였습니다.

| 항목          | 값          |
|---------------|--------------|
| 분석 기간     | 2024-07-22 ~ 2025-07-21 |
| 최고가        | 86,100원    |
| 최저가        | 49,900원    |
| 평균 종가     | 60,088.38원 |
| 최근 5일 종가 |              |
| 2025-07-15    | 63,700원    |
| 2025-07-16    | 64,700원    |
| 2025-07-17    | 66,700원    |
| 2025-07-18    | 67,100원    |
| 2025-07-21    | 67,800원    |

### 분석 요약:
1. **주가 변동성**: 
   - 최고가는 86,100원, 최저가는 49,900원으로, 주가의 변동폭이 상당히 큽니다. 이는 시장의 불확실성이나 외부 요인에 의해 영향을 받았을 가능성이 있습니다.

2. **평균 종가**: 
   - 평균 종가는 60,088.38원으로, 최근 5일 종가의 평균(약 65,100원)보다 낮습니다. 이는 최근 주가가 평균보다 상승세에 있음을 나타냅니다.

3. **최근 5일 주가 흐름**: 
   - 최근 5일 동안 주가는 지속적으로 상승세를 보였습니다. 특히, 2025-07-15일 63,700원에서 시작하여 2025-07-21일 67,800원으로 상승했습니다. 이는 긍정적인 시장 반응이나 기업 실적 개선 등의 요인으로 해석될 수 있습니다.

### 결론:
삼성전자의 주가는 최근 1년 동안 큰 변동성을 보였으나, 최근 5일 동안은 상승세를 이어가고 있습니다. 앞으로의 주가 흐름에 대한 긍정적인 전망이 가능할 것으로 보입니다.


##### 실습: 사용자가 입력한 질문에 맞는 캠핑장을 검색해서 추천하는 ChatGPT 어플리케이션을 작성하시오(힌트: 고캠핑 API) -> 10~30분 필요 (가이드에 따름)

In [25]:
import os
import requests
import openai

# API 키 설정
camp_api_key = "2/nZ3WtnwuFX4Fq9XZVkyXI+s1S7i4siht8xQRrlqZyDeXMAXUMQRsvhcApj90hMUPORLCrFfcRLqsPgf2YA5Q=="
openai.api_key = os.getenv("OPENAI_API_KEY")

# 캠핑장 검색 함수
def search_camping(keyword):
    url = "http://apis.data.go.kr/B551011/GoCamping/basedList"
    params = {
        "ServiceKey": camp_api_key,
        "MobileOS": "ETC",
        "MobileApp": "GoCampingApp",
        "_type": "json",
        "numOfRows": 5,
        "pageNo": 1,
        "keyword": keyword
    }
    res = requests.get(url, params=params).json()
    items = res.get("response", {}).get("body", {}).get("items", {}).get("item", [])
    return "\n".join(f"{i+1}. {x['facltNm']} - {x['addr1']}" for i, x in enumerate(items)) or "결과가 없습니다."

# GPT에게 캠핑장 추천 요청
def ask_gpt(question, camp_list):
    gen = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"다음 캠핑장 목록을 참고해서 '{question}'에 맞는 추천을 해줘.\n{camp_list}"},
            {"role": "user", "content": question}
        ],
        temperature=0,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stream=True
    )
    for chunk in gen:
        delta = chunk.choices[0].delta
        if delta.content:
            print(delta.content, end='')

# 실행
if __name__ == "__main__":
    q = input("💬 캠핑 질문 입력: ")
    camp_list = search_camping(q.replace("추천", "").replace("해줘", "").strip())
    ask_gpt(q, camp_list)


바다가 보이는 캠핑장은 아름다운 경치와 함께 해양 활동을 즐길 수 있는 좋은 장소입니다. 다음은 바다가 보이는 캠핑장 몇 곳을 추천합니다:

1. **해운대 해수욕장 캠핑장 (부산)** - 부산의 대표적인 해수욕장인 해운대에서 바다를 바라보며 캠핑을 즐길 수 있습니다. 다양한 편의시설과 해양 스포츠도 즐길 수 있습니다.

2. **경포대 캠핑장 (강릉)** - 경포대 근처에 위치한 캠핑장으로, 아름다운 경포해변을 감상하며 캠핑할 수 있습니다. 해변에서 수영이나 서핑도 가능합니다.

3. **송정 해수욕장 캠핑장 (부산)** - 송정 해수욕장은 서핑으로 유명한 곳으로, 바다를 가까이에서 느끼며 캠핑을 즐길 수 있습니다.

4. **동해안 캠핑장 (강원도)** - 동해안에는 여러 캠핑장이 있으며, 대부분 바다와 가까운 위치에 있습니다. 특히 맑은 바다와 아름다운 일출을 감상할 수 있습니다.

5. **제주도 캠핑장** - 제주도에는 바다를 바라보며 캠핑할 수 있는 다양한 장소가 있습니다. 특히 협재 해수욕장 근처의 캠핑장은 아름다운 경치를 자랑합니다.

이 외에도 많은 바다가 보이는 캠핑장이 있으니, 원하는 지역과 시설에 맞춰 선택하시면 좋습니다!